In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="llama3.1:8b", temperature=0)

## Write to state

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='d38fa9ea-f0a9-4ce2-910c-c08856d13feb'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T18:19:48.6146319Z', 'done': True, 'done_reason': 'stop', 'total_duration': 43807581357, 'load_duration': 17794474730, 'prompt_eval_count': 171, 'prompt_eval_duration': 19997296250, 'eval_count': 21, 'eval_duration': 5961975331, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b99af-0ce1-7463-b65b-5ff189cf5504-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '2c6cbe12-5b23-43ef-a073-d32d5ab6a713', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 171, 'output_tokens': 21, 'total_tokens': 192}),
              ToolMessage(content='Successfully updated favourite colou

In [8]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='db89f20a-567a-4bed-9a41-4d1124651a37'),
              AIMessage(content='I\'m not able to provide a response as the given functions do not seem to be related to answering questions like "Hello, how are you?" The provided functions appear to be related to updating user state in some way. \n\nHowever, if I had to choose a function that could potentially be used to respond to this question, it would be an empty string for the favourite_colour parameter of the update_favourite_colour function.\n\n{"name": "update_favourite_colour", "parameters": {"favourite_colour": ""}}', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T18:21:16.449423876Z', 'done': True, 'done_reason': 'stop', 'total_duration': 43865627369, 'load_duration': 130593143, 'prompt_eval_count': 172, 'prompt_eval_duration': 12896211828, 'eval_count': 10

## Read state

In [9]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [10]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='055f0066-0d32-4826-bcf7-8fe1b5dbb23c'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T18:21:29.262170456Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12770835796, 'load_duration': 168961683, 'prompt_eval_count': 209, 'prompt_eval_duration': 6613259591, 'eval_count': 21, 'eval_duration': 5935627203, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b99b1-0f4a-7853-95de-48a8c51d1b8d-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '25f9c69e-7b70-4435-b5a6-e1426930c801', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 209, 'output_tokens': 21, 'total_tokens': 230}),
              ToolMessage(content='Successfully updated favourite colour

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='055f0066-0d32-4826-bcf7-8fe1b5dbb23c'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-07T18:21:29.262170456Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12770835796, 'load_duration': 168961683, 'prompt_eval_count': 209, 'prompt_eval_duration': 6613259591, 'eval_count': 21, 'eval_duration': 5935627203, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b99b1-0f4a-7853-95de-48a8c51d1b8d-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '25f9c69e-7b70-4435-b5a6-e1426930c801', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 209, 'output_tokens': 21, 'total_tokens': 230}),
              ToolMessage(content='Successfully updated favourite colour